# Creating a feature matrix from a DB Query

In [1]:
import sys
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [2]:
#load in data base
db = pd.read_csv('/Users/shayneufeld/GitHub/mouse_bandit/session_record.csv',index_col=0)

In [27]:
db[db['Session ID']=='03262017_K13']

,Session ID,Mouse ID,Date,Phase,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration


### Query all 80-20 sessions where performance > 0.7 and block structure was 50

In [3]:
conditions = ['100-0','90-10','80-20','70-30']
probs = [1,0.9,0.8,0.7]

for condition,prob in zip(conditions,probs):
    print(condition)
    print(prob)

100-0
1
90-10
0.9
80-20
0.8
70-30
0.7


In [6]:
master_matrix = np.zeros(1)

conditions = ['100-0','90-10','80-20','70-30']
probs = [1,0.9,0.8,0.7]

for condition,prob in zip(conditions,probs):
    r = db[((db['Left Reward Prob'] == prob) |  (db['Right Reward Prob'] == prob))].copy()
    r = r[r['p(high Port)'] > prob-0.1].copy()
    r = r[r['Block Range Min'] == 50].copy()
    r['Condition'] = condition
    session_names = r['Session ID'].values
    conditions_ = r['Condition'].values
    
    '''
    load in trial data
    '''
    columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given']

    root_dir = '/Users/shayneufeld/GitHub/mouse_bandit/data/trial_data'

    trial_df = []

    #load in trials
    for session in session_names:
        full_name = session + '_trials.csv'

        path_name = os.path.join(root_dir,full_name)

        trial_df.append(pd.read_csv(path_name,names=columns))

    mouse_ids = r['Mouse ID'].values
    
    
    #create feature matrix
    for i,df in enumerate(trial_df):
    
        curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
        curr_feature_matrix['Condition'] = condition

        if master_matrix.shape[0]==1:
            master_matrix = curr_feature_matrix.copy()
        else:
            master_matrix = master_matrix.append(curr_feature_matrix)


In [17]:
master_matrix.index = np.arange(master_matrix.shape[0])

master_matrix[master_matrix['Session ID']=='03242017_K13'].index.values

array([158570, 158571, 158572, 158573, 158574, 158575, 158576, 158577,
       158578, 158579, 158580, 158581, 158582, 158583, 158584, 158585,
       158586, 158587, 158588, 158589, 158590, 158591, 158592, 158593,
       158594, 158595, 158596, 158597, 158598, 158599, 158600, 158601,
       158602, 158603, 158604, 158605, 158606, 158607, 158608, 158609,
       158610, 158611, 158612, 158613, 158614, 158615, 158616, 158617,
       158618, 158619, 158620, 158621, 158622, 158623, 158624, 158625,
       158626, 158627, 158628, 158629, 158630, 158631, 158632, 158633,
       158634, 158635, 158636, 158637, 158638, 158639, 158640, 158641,
       158642, 158643, 158644, 158645, 158646, 158647, 158648, 158649,
       158650, 158651, 158652, 158653, 158654, 158655, 158656, 158657,
       158658, 158659, 158660, 158661, 158662, 158663, 158664, 158665,
       158666, 158667, 158668, 158669, 158670, 158671, 158672, 158673,
       158674, 158675, 158676, 158677, 158678, 158679, 158680, 158681,
      

In [19]:
m=master_matrix.copy()

In [ ]:
m = m.drop(master_matrix[master_matrix['Session ID']=='03242017_K13'].index.values)

In [28]:
master_matrix = m.copy()

## Save combined feature matrix  

In [29]:
master_matrix.to_csv(os.path.join(root_dir,'master_data.csv'))

In [9]:
root_dir

'/Users/shayneufeld/GitHub/mouse_bandit/data/trial_data'